# POS tagger

# !!! THIS CODE WRITES INTO "data/tagged" FOLDER !!!

### IF YOU ALREADY HAVE A FOLDER NAMED "tagged" INTO "data" PLEASE MOVE ITS CONTENT INTO ANOTHER FOLDER BEFORE YOU RUN THIS NOTEBOOK

### IF YOU HAVE NOT A FOLDER NAMED "tagged" IN THE "data" FOLDER PLEASE CREATE "tagged" FOLDER INSIDE "data" FOLDER BEFORE YOU RUN THIS NOTEBOOK

## Comunque troppo coatto scrive in inglese

## Imports

In [1]:
import nltk
import json
import re
import os
import numpy as np
import random

#### if not done yet, you need to download the nltk base

In [2]:
#run command below. you just need to do it once
#nltk.download()

## Support functions

In [3]:
#accepted tags
accepted = ['CA','CD','NN','NNP']
#unwanted characters. it is a regex. they'll be substitued with space charachter
unwntd = '[/_\-|]'
#unwanted name attributes
unwntd_na = ['<page title>','url','URL','Url']

In [4]:
def pos_filtr(sentence):
    filtered = []
    subtracted = re.sub(unwntd, ' ', sentence)
    tokens = nltk.word_tokenize(subtracted)
    tagged = nltk.pos_tag(tokens)
    for elem in tagged:
        wd = elem[0]
        tg = elem[1]
        if (tg in accepted):
            filtered.append(wd)
    return filtered

In [5]:
def rndm_filtr(value):
    filtered = []
    if type(value) is 'List':
        value_strng = ' '.join(str(value))
    value_strng = str(value)
    filtered = pos_filtr(value_strng)
    if len(filtered) > 3:
        nw_filtered = []
        while (len(nw_filtered) <= 3 and filtered):
            rndm_chc = random.choice(filtered)
            filtered.remove(rndm_chc)
            if (rndm_chc not in nw_filtered):
                nw_filtered.append(rndm_chc)
        return nw_filtered
    return filtered

In [6]:
def site_prod_name(complete_path, prod_name):
    site_prod = (str(complete_path)).replace('data/specs/','')
    site_prod += "//"
    site_prod += str(prod_name)
    return site_prod

## File reader and POS tagger

In [7]:
specs_dir = os.path.join("data", "specs")
tagged_dir = os.path.join("data", "tagged")

In [16]:
i = 0
tagged = {}
for root_dir, _, files in os.walk(specs_dir):
    for file_name in files:
        if (i % 1000 == 0):
            print('read and POS-tagged ' + str(i) + ' files.')
        if file_name.endswith(".json"):
            file_path = os.path.join(root_dir, file_name)
            #print('read ' + str(file_path))
            with open(file_path, encoding='utf-8') as json_file:
                data = json.load(json_file)
                for att_name, att_value in data.items():
                    if(att_name in unwntd_na):
                        continue
                    head, parent_dir = os.path.split(root_dir)
                    original_name = parent_dir + '//' + file_name[:-5] + '//' + att_name    
                    new_att_name = ' '.join(pos_filtr(str(att_name)))
                    new_att_value = ' '.join(rndm_filtr(att_value))
                    if (new_att_name and new_att_value):
                        tagged[original_name] = [new_att_name, new_att_value]
                i += 1
path_str = str(file_path)
str_file = ('tagged.json')
tagged_file = os.path.join(tagged_dir, str_file)
with open(tagged_file, 'w', encoding='utf-8') as outpt_fl:
    json.dump(tagged, outpt_fl, ensure_ascii=False)
print('done')    

read and POS-tagged 0 files.
done
